## Resampling using EWA


In [40]:
import numpy as np
import xarray as xr
import h5py
import rasterio

from rasterio.io import DatasetWriter
from rasterio.transform import from_origin
from pyresample import kd_tree, geometry, create_area_def
import matplotlib.pyplot as plt
from pyresample.ewa import DaskEWAResampler


In [76]:
name = '2A.GPM.Ku.V9-20211125.20230101-S013441-E030713.050239.V07A.hdf5'

filename = f'/home/aniket/aniket-backup/anthromet/imerg-data/ku/{name}'

In [136]:
data = h5py.File(filename, 'r')

longitude = data['/FS/Longitude'][:]
latitude = data['/FS/Latitude'][:]
elevation = data['/FS/PRE/elevation'][:]

resolution = 0.01

In [79]:
longitude.shape

(7933, 49)

In [137]:
crs = rasterio.crs.CRS.from_epsg(4326)

swath_def = geometry.SwathDefinition(lons=longitude, lats=latitude)

area_def = create_area_def('world',
                           crs,
                           area_extent=[-180, -70, 180, 70],
                           resolution=resolution)

transform = from_origin(-180, 70, resolution, resolution)

In [138]:
resampler = DaskEWAResampler(swath_def, area_def)


In [139]:
rows_per_scan = 7933
result = resampler.resample(elevation, rows_per_scan=rows_per_scan, weight_distance_max=5000, weight_delta_max=0)

result

array([[nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]], dtype=float32)

In [140]:
output_file = 'ewa_24.tif'

with rasterio.open(output_file, 'w', driver='GTiff', height=result.shape[0], width=result.shape[1],
                   count=1, dtype=result.dtype, crs=crs, transform=transform, nodata=np.nan, compress='lzw') as dst:
    dst.write(result, 1)